In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.getOrCreate()
spark

25/10/02 19:11:35 WARN Utils: Your hostname, ehab resolves to a loopback address: 127.0.1.1; using 192.168.125.130 instead (on interface ens33)
25/10/02 19:11:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/02 19:11:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
##to display notebook cell with horizontal scroll bar
from IPython.display import display, HTML
display(HTML("<style>pre {white-space: pre !important; }</style>"))

In [4]:
df=spark.read.text('/home/ahmed/Desktop/Spark Course data/log data/access.log',lineSep='\n')

In [5]:
df.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "G

In [14]:
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.types import IntegerType

In [18]:
LOG_PATTERN = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] \"(\S+) (\S+) (\S+)\" (\d{3}) (\S+) \"([^\"]*)\" \"([^\"]*)\"(\s.*)?$'

In [19]:
extracted_logs_df = df.select(
    # 1. Client_Ip (Group 1)
    regexp_extract(col('value'),LOG_PATTERN, 1).alias('Client_Ip'),
    
    # 2. timestamp (Group 4)
    regexp_extract(col('value'), LOG_PATTERN, 4).alias('timestamp'),
    
    # 3. method (Group 5)
    regexp_extract(col('value'), LOG_PATTERN, 5).alias('method'),
    
    # 4. path (Group 6)
    regexp_extract(col('value'),LOG_PATTERN, 6).alias('path'),
    
    # 5. protocol (Group 7)
    regexp_extract(col('value'), LOG_PATTERN, 7).alias('protocol'),
    
    # 6. statusCode (Group 8) - Cast to Integer
    regexp_extract(col('value'), LOG_PATTERN, 8).cast(IntegerType()).alias('statusCode'),
    
    # 7. content-size (Group 9)
    regexp_extract(col('value'), LOG_PATTERN, 9).alias('content-size'),
    
    # 8. referrer (Group 10)
    regexp_extract(col('value'), LOG_PATTERN, 10).alias('referrer'),
    
    # 9. user_agent (Group 11)
    regexp_extract(col('value'), LOG_PATTERN, 11).alias('user_agent')
)

In [20]:
extracted_logs_df.show(5, truncate=False)

+-------------+--------------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+----------+------------+-----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------+
|Client_Ip    |timestamp                 |method|path                                                                                                                                                                  |protocol|statusCode|content-size|referrer                           |user_agent                                                                                                                                    |
+-------------+--------------------------+------+---------------------------------------------------------------------------------------------

In [22]:
from pyspark.sql.functions import to_timestamp, col

In [23]:
logs_clean=extracted_logs_df.withColumn('datetime',to_timestamp('timestamp', 'dd/MMM/yyyy:HH:mm:ss Z')).drop('timestamp')

In [29]:
logs_clean=logs_clean.withColumn('Content_Size',logs_clean["content-size"].cast("int")).drop('content-size')

In [30]:
logs_clean=logs_clean.select('Client_Ip','datetime','method','path','protocol','statusCode','Content_Size','referrer','user_agent')

In [31]:
logs_clean.printSchema()

root
 |-- Client_Ip: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- path: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- statusCode: integer (nullable = true)
 |-- Content_Size: integer (nullable = true)
 |-- referrer: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [32]:
logs_clean.count()

10365152

In [38]:
logs_filtered = logs_clean.filter(
    (col("Client_Ip").isNotNull()) & (col("Client_Ip") != "") &
    (col("method").isNotNull()) & (col("method") != "") &
    (col("statusCode").isNotNull())
)

In [39]:
logs_filtered.count()

10364866

In [40]:
logs_filtered2=logs_filtered.filter(
    (col('Content_Size')>0) & (col('Content_Size')<10000000) 
)

In [41]:
logs_filtered2.count()

9594260

In [50]:
from pyspark.sql.functions import when


In [51]:

logs_filtered3 = logs_filtered2.withColumn(
    "referrer",
    when(col("referrer") == "-", None).otherwise(col("referrer"))
)


In [52]:
logs_filtered3.printSchema()

root
 |-- Client_Ip: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- path: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- statusCode: integer (nullable = true)
 |-- Content_Size: integer (nullable = true)
 |-- referrer: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [ ]:
logs_filtered3.write.format('parquet').mode('overwrite').save('/home/ahmed/Desktop/Spark Course data/Logs_Parquet')

[Stage 30:====>                                                    (2 + 2) / 27]